#Importing Necessary Libraries 

In [0]:
import os, json
import glob
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from zipfile import ZipFile
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset
import tensorflow as tf

In [0]:
from google.colab import drive

In [3]:
from google.colab import files
uploaded = files.upload()

Saving Weibo.txt to Weibo.txt


In [4]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls drive


'My Drive'


#Creating Dataset

Since our original dataset is in JSON format, we need to convert it into a readible format pandas dataframe. This small segment below will allow us to do just that

In [0]:
#create an empty-list
weibo = []

#using ziplefile package to unpackage zipped JSON files
zf = ZipFile('/content/drive/My Drive/Weibo.zip')

#convert to text files
text_files = zf.namelist()

#for-loop to read an convert each JSON file and add to pandas dataframe
for text_file in text_files:
   data = pd.read_json(zf.open(text_file))
   weibo.append(data)

#concatenate text files indo dataframe
weibo = pd.concat(weibo)
 

In [7]:
 weibo.head()

,reposts_count,uid,bi_followers_count,text,user_description,user_avatar,id,city,province,friends_count,user_location,mid,verified_reason,attitudes_count,followers_count,verified_type,original_text,picture,statuses_count,parent,verified,geo,user_created_at,favourites_count,screen_name,gender,user_geo_enabled,username,comments_count,t
0,267,1760369714,322,毛新宇 深情演唱新作《献给爷爷奶奶的歌》 http://t.cn/h1R0Rj,本博所有帖子均来源于网络，若侵权请告知~谢谢关注。,http://tp3.sinaimg.cn/1760369714/180/1294234453/1,10031080900,1000,400,1273,海外,2111105031607336,,0,1011418,-1,毛新宇 深情演唱新作《献给爷爷奶奶的歌》 http://t.cn/h1R0Rj,NaN,116681,NaN,False,NaN,2010-06-10 15:44:15,6,历史震惊你,m,True,历史震惊你,169,1304404747
1,0,1838774143,172,我工作，大家先看个乐儿吧，看我有感觉的请自觉汇报感受。,给我一个支点，我能翘起二郎腿。,http://tp4.sinaimg.cn/1838774143/180/5664672365/0,10031139424,1,11,1946,北京 东城区,5602376064817144832,,0,745,-1,我工作，大家先看个乐儿吧，看我有感觉的请自觉汇报感受。,NaN,7161,2.111105e+15,False,NaN,2010-10-25 10:20:53,11293,冒场的冒,f,True,冒场的冒,0,1304404825
2,0,1877330775,29,[汗],,http://tp4.sinaimg.cn/1877330775/180/5713662294/0,10031149936,1,11,77,北京 东城区,5602376120817609728,,0,292,-1,[汗],NaN,2327,2.111105e+15,False,NaN,2010-11-29 02:22:16,164,cecilia1019_chen,f,True,cecilia1019_chen,0,1304404838
3,0,1607507894,259,笑而不语。。。。,东张西望。一无所长。四体不勤。五谷不分。。。大梦一场。,http://tp3.sinaimg.cn/1607507894/180/5701479239/0,10031160544,1,43,735,湖南 长沙,5602376180947154944,,0,1364,-1,笑而不语。。。。,NaN,3872,2.111105e+15,False,NaN,2010-12-03 02:42:21,14,刘小姐胖得被人打,f,True,刘小姐胖得被人打,0,1304404852
4,0,1043857555,89,在我看来，力推毛新宇，就是打压太子党的一种温柔方式。,,http://tp4.sinaimg.cn/1043857555/180/1278895325/1,10031160846,1,31,278,上海 黄浦区,5602376185242122240,,0,193,-1,在我看来，力推毛新宇，就是打压太子党的一种温柔方式。,NaN,546,2.111105e+15,False,NaN,2010-04-20 17:21:02,5,孙彦亦,m,False,孙彦亦,1,1304404853


#Exploratory Data Analysis 

In [8]:
weibo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3805656 entries, 0 to 608
Data columns (total 30 columns):
reposts_count         int64
uid                   int64
bi_followers_count    int64
text                  object
user_description      object
user_avatar           object
id                    int64
city                  int64
province              int64
friends_count         int64
user_location         object
mid                   int64
verified_reason       object
attitudes_count       int64
followers_count       int64
verified_type         int64
original_text         object
picture               object
statuses_count        int64
parent                float64
verified              bool
geo                   object
user_created_at       datetime64[ns]
favourites_count      int64
screen_name           object
gender                object
user_geo_enabled      bool
username              object
comments_count        int64
t                     int64
dtypes: bool(2), datetime64[ns]

In [0]:
weibo.describe()

,reposts_count,uid,bi_followers_count,id,city,province,friends_count,mid,attitudes_count,followers_count,verified_type,statuses_count,parent,favourites_count,comments_count,t
count,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.805656e+06,3.800992e+06,3.805656e+06,3.805656e+06,3.805656e+06
mean,2.540527e+00,2.604696e+09,1.111895e+02,3.627244e+15,2.172112e+02,5.610215e+01,3.726101e+02,5.330037e+16,1.411159e+00,1.817272e+04,4.201934e+01,4.644336e+03,4.293130e+16,3.057142e+02,8.876031e-01,1.394419e+09
std,1.805378e+02,1.200170e+09,2.372029e+02,5.236409e+14,4.067502e+02,7.476520e+01,4.500370e+02,4.874384e+17,3.845409e+02,6.948063e+05,8.717965e+01,1.369328e+04,4.192295e+17,3.547998e+03,6.711213e+01,4.555677e+07
min,0.000000e+00,1.002900e+04,-5.690000e+02,4.010313e+09,1.000000e+00,1.100000e+01,0.000000e+00,2.111101e+12,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,2.111104e+13,0.000000e+00,0.000000e+00,1.291221e+09
25%,0.000000e+00,1.794857e+09,1.100000e+01,3.516887e+15,1.000000e+00,3.100000e+01,1.180000e+02,3.521622e+15,0.000000e+00,6.900000e+01,-1.000000e+00,4.450000e+02,3.521045e+15,2.000000e+00,0.000000e+00,1.353975e+09
50%,0.000000e+00,2.208946e+09,4.100000e+01,3.600441e+15,5.000000e+00,3.700000e+01,2.200000e+02,3.604401e+15,0.000000e+00,2.050000e+02,-1.000000e+00,1.485000e+03,3.603164e+15,1.800000e+01,0.000000e+00,1.373896e+09
75%,0.000000e+00,2.922571e+09,1.100000e+02,3.911259e+15,1.800000e+01,5.100000e+01,4.220000e+02,3.911362e+15,0.000000e+00,4.770000e+02,-1.000000e+00,4.496000e+03,3.911259e+15,1.290000e+02,0.000000e+00,1.448000e+09
max,1.036820e+05,5.828321e+09,6.880000e+03,3.931344e+15,1.000000e+03,4.000000e+02,5.000000e+03,5.629141e+18,5.926620e+05,1.529447e+08,2.200000e+02,1.039647e+06,5.626076e+18,7.866530e+05,3.810300e+04,1.452789e+09


Now that we have our dataset imported we can do the same with our labels from the paper. This dataset is as follows: 

 ***This corpus contains 4664 labeled events in total. Each line contains one event with ids of relevent posts with format: event_id, label, post_ids. For the labels, the value is 1 if the event is a rumor, and is 0 otherwise.***
 
  *Note that we do not explicitly produce claim for each Weibo event since we directly collect the events as propagation threads from Sina community management center, and the source post (the first post) of each event can be considered as the main claim*

In [9]:
weibo_labels = pd.read_csv('Weibo.txt', header = None, delimiter="\t", names=["id", "label", "post_ids"])

weibo_labels['id'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')
weibo_labels['label'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'')

weibo_labels['id'] = weibo_labels['id'].astype(int)
weibo_labels['post_ids'] = weibo_labels['post_ids'].astype(str)
weibo_labels['post_ids'] = weibo_labels['post_ids'].str.split()
weibo_labels

,id,label,post_ids
0,10031080900,0,"[10031080900, 10031139424, 10031149936, 100311..."
1,10031994215,0,"[10031994215, 10032008166, 10032010844, 100320..."
2,10276391917,0,"[10276391917, 10276408777, 10276417123, 102764..."
3,10313557537,0,"[10313557537, 10313678159, 10313758223, 103137..."
4,10402071863,0,"[10402071863, 10402138931, 10402135371, 104021..."
...,...,...,...
4659,3585254264847900,1,"[3585254264847900, 3585254944578740, 358525494..."
4660,3585254046272685,1,"[3585254046272685, 3585254218699522, 358525434..."
4661,3585553637175264,1,"[3585553637175264, 3585553859199477, 358555399..."
4662,3585553678828854,1,"[3585553678828854, 3585553830129434, 358555383..."


Now we need to re-arrange the dataframe into a list for our model implementation, following the original paper

In this dataset above, post_ids represents our propagation path by identifying the user ids who engaged in propagating the news. 

This sequence as mentioned in the paper is a a variable-length multivariate time series. Therefore, we need to be able to truncated this sequence with a sequence limit or length denoted by *sequence_length* will represent this truncation

In [0]:
# since most of our propagation paths are more than 100 sequences long, we need to create a limit to preserve sotrage capacities
sequence_length = 10
post_ids = weibo_labels['post_ids']
post_ids = post_ids.values.tolist()

Now we are going to create a function definition where we will set a limit for the length of each path sequence. For the sake of convinience, we will cap each path to 10 user posts

In [12]:
#capping of the sequence length of each list in the array to 10 as per the sequence_length set in the original paper
def ListCap(list_of_lists,new_LoL):
  
  for list in list_of_lists:

    list = list[:10]
    new_LoL.append(list)

  return new_LoL

paths = list()
paths = ListCap(post_ids,paths)

print(paths[1])


['10031994215', '10032008166', '10032010844', '10032017004', '10032019110', '10032021194', '10032024018', '10032024524', '10032024896', '10032028328']


Note that the first id in each sample's post_id column is the same as the id of the corresponding event_id, thus indicating its original propagation path

#Pre-Processing

First, we need to extract the necessary features mentioned in Table 2 of the paper from the original dataset

In [0]:
# creating a new dataset using required extracted features from original dataset 
new_weibo = weibo[['id','user_description', 'username', 'followers_count'
                   , 'friends_count', 'statuses_count', 'user_created_at'
                   , 'verified', 'user_geo_enabled']].copy()

Since the paper needs the length of each sample username and description we need to convert the Cantonese-baed text to a number. Therefore, for this exercise, we will assume that each character represents a signle character in a string and use that to count the length of each 

In [0]:
# convert the user_description to a character counter integer value
new_weibo['user_description']  = new_weibo['user_description'].str.len()

# convert the username to a character counter integer value
new_weibo['username']  = new_weibo['username'].str.len()

Now we need to convert the 'user_created_at' coloumn to a resitration_ag using the first date of the conference this paper is for. Since the first day of the conference was February 2, 2018 we will use that as the starting date and subtract the difference within each cell of the column  

In [0]:
conference_date = pd.Timestamp('2018-02-02')

new_weibo['registration_age'] = (conference_date - new_weibo['user_created_at']).dt.days
weibo = new_weibo.drop(columns ='user_created_at')

Now our final step is to implement a conversion of our catagorical variables and normalize the features so that our model can converge faster later on in this model

In [0]:
verified = pd.get_dummies(weibo['verified'], prefix='verified')
geo_enabled = pd.get_dummies(weibo['user_geo_enabled'], prefix='user_geo_enabled')

weibo = pd.concat([weibo, verified, geo_enabled], axis=1)

pre_norm_weibo = weibo.drop(columns =['verified','user_geo_enabled'])


In [0]:
x = pre_norm_weibo[['user_description', 'username', 'followers_count', 
                    'friends_count', 'statuses_count', 'registration_age',
                    'verified_False','verified_True', 'user_geo_enabled_False',
                   'user_geo_enabled_True']]

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)

pre_norm_weibo[['user_description', 'username', 'followers_count', 
                    'friends_count', 'statuses_count', 'registration_age',
                    'verified_False','verified_True', 'user_geo_enabled_False',
                   'user_geo_enabled_True']] = x_scaled

In [18]:
final_weibo = pre_norm_weibo

final_weibo

,id,user_description,username,followers_count,friends_count,statuses_count,registration_age,verified_False,verified_True,user_geo_enabled_False,user_geo_enabled_True
0,10031080900,0.030902,0.15625,0.006613,0.2546,0.112231,0.871904,1.0,0.0,0.0,1.0
1,10031139424,0.018541,0.12500,0.000005,0.3892,0.006888,0.813407,1.0,0.0,0.0,1.0
2,10031149936,0.000000,0.50000,0.000002,0.0154,0.002238,0.798463,1.0,0.0,0.0,1.0
3,10031160544,0.033375,0.25000,0.000009,0.1470,0.003724,0.796755,1.0,0.0,0.0,1.0
4,10031160846,0.000000,0.09375,0.000001,0.0556,0.000525,0.893681,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
604,12284438762,0.000000,0.18750,0.000001,0.0338,0.000177,0.768147,1.0,0.0,1.0,0.0
605,13418764769,0.065513,0.21875,0.000006,0.0818,0.006930,0.829633,1.0,0.0,0.0,1.0
606,3502616032013968,0.014833,0.12500,0.000002,0.0452,0.000188,0.590521,1.0,0.0,0.0,1.0
607,3502794348214954,0.000000,0.28125,0.000003,0.0322,0.005327,0.887703,1.0,0.0,0.0,1.0


In [19]:
weibo = pd.merge(final_weibo, weibo_labels, on='id')
weibo = weibo.drop(['post_ids','id'], axis = 1)
weibo['label'] = weibo['label'].astype(int)
weibo.head()

,user_description,username,followers_count,friends_count,statuses_count,registration_age,verified_False,verified_True,user_geo_enabled_False,user_geo_enabled_True,label
0,0.030902,0.15625,0.006613,0.2546,0.112231,0.871904,1.0,0.0,0.0,1.0,0
1,0.021014,0.18750,0.023485,0.0500,0.055124,0.933390,1.0,0.0,1.0,0.0,0
2,0.065513,0.18750,0.071509,0.1850,0.077547,0.909906,1.0,0.0,0.0,1.0,0
3,0.040791,0.06250,0.000860,0.0398,0.000313,0.764304,0.0,1.0,1.0,0.0,0
4,0.074166,0.18750,0.007925,0.0000,0.017776,0.925705,1.0,0.0,1.0,0.0,0


In [20]:
#converting our dataframe of user characteristics into a list
features = weibo.values.tolist()
len(features)

4718

Now we have our finalized training dataset loaded and ready to use with our models. 

These features are the xj characterstic features of each user in the fixed-length multivariate sequence 

Before we begin doing that, we need to established the split of training. testing and validation percentages from this particular dataset. Based on the paper, 10% of the original *Weibo* dataset is used for validation, while the training and testing portion of the data set make a 3:1 ratio, respectively.

The following quick function definition will let us split the dataset into the appropriate sections

In [93]:
#list of propagation paths
new_paths = []

#feature counter
i = 0

# for each tweet propgation
for path in paths:

  #create a new tweet path
  new_path = []

  # for each user that contributed to that tweet propagation
  for user_id in path:

    # conditional statement to make sure we are iterating through the entire set of features we converted to a list above
    if i < len(features) and ((i % 10 != 0) or i <= 10 or i >10):

      #assign the features to that corresponding id
      user_id = features[i]
      #add those user characteristics to the list of this path
      new_path.append(user_id)
      i += 1

  #add path to the list of current paths    
  new_paths.append(new_path)

# cutting our list down for memory management
# therefore we have exactly 470 propagation paths

new_paths = new_paths[0:471]

print(new_paths[2])

[[0.044499381953028425, 0.21875, 4.6402377935854325e-05, 0.0074, 0.000491512984695767, 0.77198975234842, 1.0, 0.0, 1.0, 0.0, 0.0], [0.3485784919653893, 0.15625, 0.002777977050902068, 0.2934, 0.05255630035964131, 0.8573868488471392, 0.0, 1.0, 0.0, 1.0, 0.0], [0.09394313967861556, 0.1875, 0.0008337324535001359, 0.0064, 0.00263935739727042, 0.8198121263877027, 0.0, 1.0, 1.0, 0.0, 1.0], [0.08281829419035847, 0.09375, 0.010066288775081051, 0.547, 0.057405061525690936, 0.9940222032450896, 0.0, 1.0, 0.0, 1.0, 0.0], [0.034610630407911, 0.125, 0.011643727707472505, 0.028, 0.03712991044075538, 0.812126387702818, 1.0, 0.0, 0.0, 1.0, 1.0], [0.018541409147095178, 0.1875, 0.00010371065503994947, 0.2674, 0.0031472220859580227, 0.8569598633646456, 1.0, 0.0, 0.0, 1.0, 0.0], [0.12360939431396785, 0.125, 0.0016883417271706513, 0.0538, 0.00804022903927968, 0.9829205807002561, 0.0, 1.0, 0.0, 1.0, 0.0], [0.0630407911001236, 0.15625, 0.012995196147565656, 0.007, 0.04475557569059498, 0.7527754056362084, 1.0, 

# Converting The List of Lists into Tuple of Tuples 

Now we need to convert it into a format where we can convert each path into a dataframe

In [134]:
#create a list of dataframes
df_list = list()
i = 0

#for each path we will convert the path into a tuple
for new_path in new_paths:

  #for each user in the current path
  for user_chars in new_path:
    
    new_paths = tuple(tuple(tuple(user_chars) for user_chars in new_path) for new_path in new_paths)
  
  if i < len(new_paths):

    df_list.append(pd.DataFrame(new_paths[i]))
    i += 1

#combine each dataframe into a single dataframe
df = np.stack(df_list)
df.shape
print(df[1,:,:10])

[[4.57354759e-02 3.43750000e-01 6.79918738e-05 2.44000000e-02
  1.06767008e-04 7.32707088e-01 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [1.97775031e-02 2.18750000e-01 7.21926064e-03 2.78000000e-02
  4.58251695e-02 8.56959863e-01 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [5.56242274e-02 6.25000000e-02 2.46831817e-02 1.49800000e-01
  3.33574761e-03 8.04440649e-01 0.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [2.34857849e-02 1.87500000e-01 9.75044923e-04 2.02000000e-02
  1.97961423e-02 7.22886422e-01 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [6.05686032e-02 1.87500000e-01 3.93641500e-03 2.00000000e-04
  1.28293546e-02 9.44491887e-01 1.00000000e+00 0.00000000e+00
  0.00000000e+00 1.00000000e+00 0.00000000e+00]
 [4.69715698e-02 1.56250000e-01 1.05878366e-02 6.18000000e-02
  2.27836949e-02 7.89496157e-01 1.00000000e+00 0.00000000e+00
  0.00000000e+00

#Converting Into Tensor Dataset

Before we run our model on the training and testing features and labels we need to convert the pandas dataframe into a tensor dataset

#RNN-Based Propagation Path Representation

Based on the paper, the values fo the input dimensions, hidden dimensions, output dimenison, and dropout rate are as follows:

**GRU output dim: 32**

**Dropout rate: 0.5**

### GRU Model Class Definition

As done in the paper, we will begin with defining our Recurrent Neural Network architecture, a Gated-Recurrent Unit (GRU).

This model will be introduced using a *'class'* object with multiple definitions inside:


The **init** function definition represents the initialization of our GRU paramters defined from the paper and implemented into the PyTorch NN package 

In [0]:
class RNNEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(RNNEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_encoder = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)

    def forward(self, x: torch.Tensor, h_0: torch.Tensor)->torch.Tensor:
        # x.shape: Batch x sequence length x input_size
        # h_0.shape:
        # output.shape: batch x sequence length x input_size
        output, h_n = self.rnn_encoder(x, h_0)

        # output.shape: batch x input_size
        output = output.mean(dim=1)
        return output


#CNN-Based Propagation Path Representation 

In [0]:
class CNNEncoder(nn.Module):
  
    def __init__(self, out_channels: int, kernel_size: tuple):
        super(CNNEncoder, self).__init__()
        self.cnn_encoder = nn.Conv2d(in_channels=1, out_channels=out_channels, kernel_size=kernel_size)

    def forward(self, x: torch.Tensor):
        # x.shape: batch x sequence length x kernel_size[1](input_size)
        # x.shape: batch x 1 x sequence length x kernel_size[1]
        x = x.unsqueeze(dim=1)
        # output.shape: batch x out_channels x sequence length - kernel_size[0] + 1
        output = F.relu(self.cnn_encoder(x))
        # output.shape: batch x out_channels
        output = output.mean(dim=2)
        return output

#Concatenating RNN & CNN Outputs

In [0]:
class DetectModel(nn.Module):
    def __init__(self, input_size,
                 hidden_size, rnn_layers,
                 out_channels, height, cnn_layers,
                 linear_hidden_size, linear_layers, output_size):
        super(DetectModel, self).__init__()
        self.rnn_encoder = RNNEncoder(input_size=input_size, hidden_size=hidden_size, num_layers=rnn_layers)
        self.cnn_encoder = CNNEncoder(out_channels=out_channels, kernel_size=(height, input_size))

        self.linear = nn.Sequential(
            nn.Linear(hidden_size + out_channels, linear_hidden_size), nn.ReLU(inplace=True),
            *chain(*[(nn.Linear(linear_hidden_size, linear_hidden_size), nn.ReLU(inplace=True)) for i in range(linear_layers - 2)]),
            nn.Linear(linear_hidden_size, output_size)
        )

    def forward(self, x, h_0):
        # h0 for rnn_encoder
        rnn_output = self.rnn_encoder(x, h_0)
        cnn_output = self.cnn_encoder(x)
        cnn_output = cnn_output.squeeze()

        # output.shape: batch x (hidden_size + out_channels)
        # output.shape: batch x output_size
        output = torch.cat([rnn_output, cnn_output], dim=1)
        output = self.linear(output)

        return output

## Defining Model Parameters and Implementation

In [0]:
from itertools import chain

In [0]:
input_size = 10
hidden_size = 5
out_channels = 32
height = 3
linear_hidden_size = 20
linear_layers = 5
output_size = 2
num_layers = 1
batch_size = 15

#Defining Loss Function and Optimizer 

In this segment, we will quickly define our model optimatizer and criterion, as well as our learning rate, which is subject to change as we experiment 

In [0]:
import torch.optim as optim

In [0]:
lr = 0.1

optimizer = optim.Adam(params=model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss(reduction='mean')
initial_hidden_state = torch.zeros(num_layers,batch_size, hidden_size, dtype=torch.float, requires_grad=False)  

#Training Our Model

In [195]:
#based on the original paper, we will iterate over 200 epochs
epochs = 200

#seperate our features and labels
X = df[:,:,:10]
y = df[:,:,:11]

#splitting our training and testing data  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

#creating our training tensors  
train_y = torch.tensor(y_train)
train_x = torch.tensor(X_train) 

#creating our testing tensors  
test_y = torch.tensor(y_test)
test_x = torch.tensor(X_test)

train = TensorDataset(train_x, train_y)
test = TensorDataset(test_x, test_y) 

#creating our training and testing loaders  
train_loader = DataLoader(dataset = train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test, batch_size = batch_size, shuffle = True)

#let pytorch know that we are training  
model.train()
 
for epoch in range(epochs):
   
  #iterate through training dataset 
  for step, (inputs,y) in enumerate(train_loader):
     
    optimizer.zero_grad()
    inputs = torch.tensor(inputs,dtype=torch.float, requires_grad=False)
    y = torch.tensor(y, dtype=torch.long, requires_grad=False)

    output = model(inputs, initial_hidden_state)

    loss = criterion(output, y)
                      
    loss.backward()

    optimizer.step()

  print("Epoch: {:3,} / Loss: {:10.5f} / Learning rate: {:.4f}".format(epoch, loss.item(), lr))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: ignored

#Testing Our Model

In [197]:
model.eval()

total_loss = []
total_output = []
total_labels = []

criterion = nn.CrossEntropyLoss(reduction='none')

for (inputs, y) in enumerate(test_loader):
  
  inputs = torch.tensor(inputs, dtype=torch.float)
  y = torch.tensor(y, dtype=torch.long)

  output = model(inputs, initial_hidden_state)
  loss = criterion(output, y)
  total_loss.append(loss)
  output = F.softmax(output, dim=1).argmax(dim=1)
  total_output.append(output)
  total_labels.append(y)

  total_loss = torch.cat(total_loss).contiguous().mean()
  total_output = torch.cat(total_output).cpu().numpy()
  total_labels = torch.cat(total_labels).cpu().numpy()

  precision, recall, f1, _ = precision_recall_fscore_support(total_labels, total_output, labels=[0, 1])



ValueError: ignored

In [0]:
x`x`